##Create a conda package with the deps for the worker nodes

Commands have to be executed in shell, doesn't work from the notebook

conda create -y -n pyspark_conda_env3 -c conda-forge conda-pack  
conda activate pyspark_conda_env3  
pip install irsx  
conda pack -f -o pyspark_conda_env.tar.gz --ignore-missing-files

In [11]:
pip install irsx

Note: you may need to restart the kernel to use updated packages.
time: 3.01 s (started: 2022-01-16 23:57:36 +00:00)


In [1]:
# To have time runtime for cells
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 538 µs (started: 2022-01-17 02:46:29 +00:00)


In [1]:
import os

os.environ['PYSPARK_PYTHON'] = "."

In [1]:
from pyspark.sql.session import SparkSession
import pandas as pd

path = "/opt/conda/miniconda3/lib/python3.8/site-packages/irsx/CSV/index_2021.csv"

df21 = pd.read_csv(path, index_col=False, dtype=str) # read all as string, not beautiful but we only need object id anyways
df21.head()
spark = SparkSession.builder.config('spark.yarn.dist.archives', "pyspark_venv.tar.gz#environment").getOrCreate()
spark


In [2]:
SparkSession.builder.config("spark.some.config.option", "some-value").getOrCreate().sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.executor.memory', '2688m'),
 ('spark.sql.warehouse.dir', 'file:/spark-warehouse'),
 ('spark.yarn.am.memory', '640m'),
 ('spark.eventLog.dir',
  'gs://dataproc-temp-us-east1-450905289299-pknkkec5/783e3fc9-4267-40f9-ab3a-517e5d75ac9b/spark-job-history'),
 ('spark.executor.instances', '2'),
 ('spark.history.fs.logDirectory',
  'gs://dataproc-temp-us-east1-450905289299-pknkkec5/783e3fc9-4267-40f9-ab3a-517e5d75ac9b/spark-job-history'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.ui.proxyBase', '/proxy/application_1642387343980_0007'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://cluster-jupyter-mw-m:8088/proxy/application_1642387343980_0007'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.sql.cbo.

In [ ]:
#file:///tmp/spark-events
import pyspark
config = pyspark.SparkConf().setAll([('spark.yarn.dist.archives', "file:///BIGData/pyspark_venv.tar.gz#environment")])

sc.stop()

sc = pyspark.SparkContext(conf=config)

22/01/17 12:09:15 ERROR org.apache.spark.deploy.yarn.ApplicationMaster: Uncaught exception: 
org.apache.hadoop.yarn.exceptions.InvalidApplicationMasterRequestException: Application doesn't exist in cache appattempt_1642420546739_0009_000001
	at org.apache.hadoop.yarn.server.resourcemanager.ApplicationMasterService.throwApplicationDoesNotExistInCacheException(ApplicationMasterService.java:361)
	at org.apache.hadoop.yarn.server.resourcemanager.ApplicationMasterService.registerApplicationMaster(ApplicationMasterService.java:259)
	at org.apache.hadoop.yarn.api.impl.pb.service.ApplicationMasterProtocolPBServiceImpl.registerApplicationMaster(ApplicationMasterProtocolPBServiceImpl.java:90)
	at org.apache.hadoop.yarn.proto.ApplicationMasterProtocol$ApplicationMasterProtocolService$2.callBlockingMethod(ApplicationMasterProtocol.java:95)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine.java:528)
	at org.apache.hadoop.ipc.RPC$Server.call(RPC.java:1086)


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Unmanaged application application_1642420546739_0010 failed due to ApplicationMaster for attempt appattempt_1642420546739_0010_000001 timed out. Failing the application.
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.waitForApplication(YarnClientSchedulerBackend.scala:97)
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.start(YarnClientSchedulerBackend.scala:64)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:220)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:579)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(Deleg

In [ ]:
spark

In [6]:
spark.stop()

In [2]:
spark.sparkContext.getConf().getAll()
spark.sparkContext._conf.getAll()  


[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.executor.memory', '2688m'),
 ('spark.app.startTime', '1642388924670'),
 ('spark.sql.warehouse.dir', 'file:/spark-warehouse'),
 ('spark.driver.port', '37453'),
 ('spark.yarn.am.memory', '640m'),
 ('spark.eventLog.dir',
  'gs://dataproc-temp-us-east1-450905289299-pknkkec5/783e3fc9-4267-40f9-ab3a-517e5d75ac9b/spark-job-history'),
 ('spark.executor.instances', '2'),
 ('spark.history.fs.logDirectory',
  'gs://dataproc-temp-us-east1-450905289299-pknkkec5/783e3fc9-4267-40f9-ab3a-517e5d75ac9b/spark-job-history'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.submit.deployMode', 'client'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://cluster-jupyter-mw-m:8088/proxy/application_1642387343980_0006'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spar

In [11]:
from pyspark.sql.types import StringType
sdf = spark.createDataFrame(df21["OBJECT_ID"], StringType())

time: 674 ms (started: 2022-01-17 02:35:18 +00:00)


In [12]:
from irsx.xmlrunner import XMLRunner
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import udf

xml_runner = XMLRunner()
def transform_data(e):
    import irsx
    from irsx.xmlrunner import XMLRunner
    filing = xml_runner.run_filing(e)
    schedules = filing.list_schedules()
    
    ein = 0
    state = 0
    name = 0
    revenue = 0
    revenueEZ = 0
    
    if "ReturnHeader990x" in schedules:
        header = filing.get_parsed_sked("ReturnHeader990x")
        header_part_i = header[0]["schedule_parts"]["returnheader990x_part_i"]
        ein = header_part_i["ein"]
        state = header_part_i.get("USAddrss_SttAbbrvtnCd", "XX")
        name = header_part_i["BsnssNm_BsnssNmLn1Txt"]
        
    if "IRS990EZ" in schedules:
        irs990ez = filing.get_parsed_sked("IRS990EZ")
        irs990ez_part_i = irs990ez[0]["schedule_parts"]["ez_part_i"]
        revenueEZ = irs990ez_part_i["TtlRvnAmt"]        
    
    if "IRS990" in schedules:
        irs990 = filing.get_parsed_sked("IRS990")
        irs990_part_i = irs990[0]["schedule_parts"]["part_i"]
        revenue = irs990_part_i["CYTtlRvnAmt"]
    
    revenue = int(revenue) + int(revenueEZ)
    return [e, ein, state, name, revenue]
     
    
my_schema = StructType([
    StructField("ObjectID", StringType(), nullable=False),
    StructField("EIN", StringType(), nullable=False),
    StructField("State", StringType(), nullable=False),
    StructField("Name", StringType(), nullable=False),
    StructField("Revenue", IntegerType(), nullable=False),
])

spark_transform_data = udf(lambda z: transform_data(z), my_schema)
spark.udf.register("spark_transform_data", spark_transform_data)

22/01/17 02:35:21 WARN org.apache.spark.sql.catalyst.analysis.SimpleFunctionRegistry: The function spark_transform_data replaced a previously registered function.


<function __main__.<lambda>(z)>

time: 159 ms (started: 2022-01-17 02:35:20 +00:00)


In [16]:
small_sdf = sdf.sample(0.0002).repartition(10) #get 72 entries
small_sdf2 = small_sdf.withColumn('valuelist', spark_transform_data('value')).select("valuelist.*")
small_sdf2.show()
#TODO write somewhere measure time

22/01/17 02:37:03 WARN org.apache.spark.scheduler.TaskSetManager: Stage 6 contains a task of very large size (5208 KiB). The maximum recommended task size is 1000 KiB.
22/01/17 02:37:04 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 8.0 (TID 14) (cluster-jupyter-mw-w-2.us-east1-c.c.fleet-parity-334009.internal executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 588, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 249, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69,

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 588, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 249, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 160, in _read_with_length
    return self.loads(obj)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 430, in loads
    ret

time: 1.7 s (started: 2022-01-17 02:37:03 +00:00)


In [18]:
transform_data("202010659349301301")

['202010659349301301', '452772761', 'NJ', "CAMDEN'S CHARTER SCHOOL", 5148336]

time: 35.5 ms (started: 2022-01-17 00:02:23 +00:00)


In [23]:
spark.list_packages()

AttributeError: 'SparkSession' object has no attribute 'list_packages'

time: 7.25 ms (started: 2022-01-17 02:12:19 +00:00)


## Start a Sparkession

In [7]:
from pyspark.sql.session import SparkSession
from pyspark import SparkContext
sc = SparkContext(appName="BigDataIRS3") #TODO use getOrCreate
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=SparkByExamples.com, master=local[1]) created by getOrCreate at /tmp/ipykernel_2197/4270203480.py:5 

## Einlesen des Files  
Das File kann entweder lokal oder mittels hdfs eingelesen werden  
Falls das File lokal eingelesen wird muss es auf jedem Node vorhanden sein, deswegen empfiehlt sich die Verwendung von HDFS 

In [ ]:
from pyspark.sql.session import SparkSession
spark2 = SparkSession.builder.appName("Test1").getOrCreate()

In [ ]:
fs_path = "file:///revenue_2021_100.csv" #local_fs
hdfs_path = "hdfs://spark-jupyter-m/user/hdfs/spark_csv/revenue_2021_100.csv" # hdfs
df = spark2.read.csv(fs_path, header=True)

type(df), df.printSchema(), df.head(5)

## Simple Aggregation
Das DF kann entweder direkt aggregiert werden, oder vorher in ein RDD umgewandelt werden

In [ ]:
!less spark_csv/revenue_2021_100.csv

In [ ]:
# Aggregation als DF
grouped_df = df.groupby("State").sum("Revenue").sort("Sum(Revenue)", ascending = False)

# Aggregation als RDD
rdd = df.rdd.drop("ObjectID", "EIN", "Name", "RevenueEZ") # U
from operator import add
reduced_rdd = rdd.reduceByKey(add).sortBy(lambda x: x[1], ascending = False)
reduced_rdd.collect()

In [ ]:
#from google.cloud import storage
#client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
#bucket = client.get_bucket('sparkbucket02')
# Then do other things...
#blob = bucket.get_blob("revenue_2021_100.csv") #('remote/path/to/file.txt')

#df = pd.read_csv(blob.download_as_string())
#df

In [ ]:
#spark = SparkSession \
#    .builder \
#    .appName("Protob Conversion to Parquet") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()\

#df = spark.read.csv('/home/hadoop/observations_temp.csv, header=True)

sudo su - hdfs  
hdfs dfsadmin -safemode leave  

hdfs dfs -mkdir spark_csv  
hdfs dfs -put /spark_csv/revenue_2021_100.csv spark_csv/revenue_2021_100.csv  
hdfs dfs -ls spark_csv  

https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
https://stackoverflow.com/questions/33055403/how-to-navigate-directories-in-hadoop-hdfs
https://stackoverflow.com/questions/28213116/hadoop-copy-a-local-file-system-folder-to-hdfs
https://stackoverflow.com/questions/61197811/can-i-read-csv-files-from-google-storage-using-spark-in-more-than-one-executor
https://groups.google.com/g/cloud-dataproc-discuss/c/cubkWrjkk2g?pli=1
https://stackoverflow.com/questions/56448009/storing-source-file-in-google-dataproc-hdfs-vs-google-cloud-storagegoogle-bucke

hdfs dfs -put /spark_csv/revenue_2021_100.csv /user/root/spark_csv/revenue_2021_100.csv